# Simple Example using Response API 

In [ ]:
# pip install --upgrade openai

In [2]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2025-03-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

response = client.responses.create(
    model="gpt-4o", # replace with your model deployment name 
    input="This is a test."
    #truncation="auto" required when using computer-use-preview model.

)
print(response.output[0].content[0].text)

# second_response = client.responses.create(
#     model="gpt-4o",  # replace with your model deployment name
#     previous_response_id=response.id,
#     input=[{"role": "user", "content": "Explain this at a level that could be understood by a college freshman"}]
# )
# print(second_response.output[0].content[0].text)


AttributeError: 'AzureOpenAI' object has no attribute 'responses'

# Chaining Responses

In [5]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2025-03-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

response = client.responses.create(  
    model="gpt-4o",  # replace with your model deployment name  
    tools=[  
        {  
            "type": "function",  
            "name": "get_weather",  
            "description": "Get the weather for a location",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "location": {"type": "string"},  
                },  
                "required": ["location"],  
            },  
        }  
    ],  
    input=[{"role": "user", "content": "What's the weather in San Francisco?"}],  
)  

print(response.model_dump_json(indent=2))  
  
# To provide output to tools, add a response for each tool call to an array passed  
# to the next response as `input`  
input = []  
for output in response.output:  
    if output.type == "function_call":  
        match output.name:  
            case "get_weather":  
                input.append(  
                    {  
                        "type": "function_call_output",  
                        "call_id": output.call_id,  
                        "output": '{"temperature": "70 degrees"}',  
                    }  
                )  
            case _:  
                raise ValueError(f"Unknown function call: {output.name}")  
  
second_response = client.responses.create(  
    model="gpt-4o",  
    previous_response_id=response.id,  
    input=input  
)  

print(second_response.model_dump_json(indent=2)) 


{
  "id": "resp_67eeae0477708190ae8371e442f242b5",
  "created_at": 1743695364.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o",
  "object": "response",
  "output": [
    {
      "arguments": "{\"location\":\"San Francisco\"}",
      "call_id": "call_v2zTPPIgENleXHPOP911swvQ",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_67eeae050a6c81909f1384021d9bc009",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          }
        },
        "required": [
          "location"
        ]
      },
      "strict": true,
      "type": "function",
      "description": "Get the weather for a location"
    }
  ],
  "top_p": 1.0,
  "max_output_tokens": null,
  "previous_resp